In [1]:
import numpy as np
import requests
from PIL import Image

In [2]:
def get_from_url(url):
    r = requests.get(url)
    path = '/tmp/image.jpeg'

    with open(path, 'wb') as f:
        f.write(r.content) 

In [3]:
def preprocess(img, scale=1./255):
    return np.float32(img*scale)

In [4]:
url = 'https://upload.wikimedia.org/wikipedia/commons/9/9a/Pug_600.jpg'
#url = 'https://www.nj.com/resizer/mg42jsVYwvbHKUUFQzpw6gyKmBg=/1280x0/smart/advancelocal-adapter-image-uploads.s3.amazonaws.com/image.nj.com/home/njo-media/width2048/img/somerset_impact/photo/sm0212petjpg-7a377c1c93f64d37.jpg'

In [5]:
get_from_url(url)

In [48]:
with Image.open('/tmp/image.jpeg') as img:
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize((150, 150), Image.NEAREST)

In [49]:
X = preprocess(np.array([np.array(img)]))
X.dtype

dtype('float32')

In [32]:
classes = ['cat', 'dog']

In [33]:
import tflite_runtime.interpreter as tflite

In [34]:
# Load model to interpreter
interpreter = tflite.Interpreter(model_path='cat_dog_classifier.tflite')
# Load weights
interpreter.allocate_tensors()

In [35]:
# Find input pointer
input_index = interpreter.get_input_details()[0]['index']

In [36]:
# Find output pointer
output_index = interpreter.get_output_details()[0]['index']

In [50]:
# Load data into input
interpreter.set_tensor(input_index, X)

In [51]:
# Run inference
interpreter.invoke()

In [52]:
# Load output into memory
lite_pred = interpreter.get_tensor(output_index)

In [53]:
dict(zip(classes, (np.array([1-lite_pred[0][0], lite_pred[0][0]])).tolist()))

{'cat': 0.04546636343002319, 'dog': 0.9545336365699768}